In [1]:
import numpy as np 

data_path = "/data/qgong/data/AE/CICDDoS2019/Data/cicddos"

X_train = np.load(data_path+"/X_train.npy")
y_train = np.load(data_path+"/y_train.npy")
X_test = np.load(data_path+"/X_test.npy")
y_test = np.load(data_path+"/y_test.npy")

X_adv = np.load(data_path+"/Adv/Adv_X_test.npy")
y_adv = np.copy(y_test)

print(np.unique(y_train))
print(np.unique(y_test))
print(np.unique(y_adv))

X_train.shape, X_test.shape, y_train.shape, y_test.shape, X_adv.shape, y_adv.shape


# In[2]:


import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import f1_score


# Parameters
input_dim = 78  # Replace with the number of features in your tabular dataset
output_dim = 11  # Replace with the number of classes


from sklearn.metrics import precision_score, recall_score, f1_score

# Define a more practical classifier architecture
def create_classifier(input_dim, output_dim):
    inputs = Input(shape=(input_dim,))
    x = Dense(128, activation='relu')(inputs)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    outputs = Dense(output_dim, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

[ 0  1  2  3  4  5  6  7  8  9 10]
[ 0  1  2  3  4  5  6  7  8  9 10]
[ 0  1  2  3  4  5  6  7  8  9 10]


2024-12-05 10:30:36.913121: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-05 10:30:36.913154: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-05 10:30:36.913182: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 10:30:36.919153: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from alibi_detect.saving import load_detector
from alibi_detect.utils.fetching import fetch_detector, fetch_tf_model


model = create_classifier(input_dim, output_dim)
model.load_weights('classifier.weights.h5')
# pretrained_classifier.compile()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)



model_path = ('adv_ae_detector_with_pretrained_classifier')
detector_type = 'adversarial'
detector_name = 'base'


ae = load_detector(model_path)

/data/qgong/miniconda3_envs/my_ae_detector/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-05 10:30:38.899515: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-05 10:30:38.899776: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-05 10:30:38.900017: I tensorflow/com

No threshold level set. Need to infer threshold using `infer_threshold`.


In [ ]:
ae.infer_threshold(X_train, threshold_perc=95, batch_size=64)

In [ ]:
predictions = ae.predict(X_adv, batch_size=64, return_instance_score=True)

In [ ]:

print(predictions['data']['instance_score'][:20])
print(predictions['data']['is_adversarial'][:20])

predictions['data']['instance_score'].shape, predictions['data']['is_adversarial'].shape


[0.0000000e+00 0.0000000e+00 4.6164505e-03 0.0000000e+00 1.6068031e+01
 3.2281112e-03 0.0000000e+00 0.0000000e+00 2.0547230e+00 1.4410389e-04]
[0 0 0 0 1 0 0 0 1 0]


((490783,), (490783,))